# Metro Nashville Disaster Analysis

#### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import HeatMap
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster
from colour import Color

#### Formatting

In [2]:
%matplotlib inline

#### Load Data

In [3]:
derecho_911calls = pd.read_csv('../data/911_Phone Calls_Derecho_050320 050420.csv')
tornado_911calls = pd.read_csv('../data/911_Phone_Calls_Tornado_030320.csv')
derecho_dispatchdata = pd.read_csv('../Dispatch_Data_Derecho_cleaned.csv')
tornado_dispatchdata = pd.read_csv('../data/Computer Aided Dispatch Data_Tornado Incidents 030320.csv', skiprows=3)

In [4]:
derecho_911calls['ALI Longitude'] = pd.to_numeric(derecho_911calls['ALI Longitude'])
derecho_911calls['ALI Latitude'] = pd.to_numeric(derecho_911calls['ALI Latitude'])
tornado_911calls['ALI Longitude'] = pd.to_numeric(tornado_911calls['ALI Longitude'])
tornado_911calls['ALI Latitude'] = pd.to_numeric(tornado_911calls['ALI Latitude'])
derecho_dispatchdata['Longitude1'] = pd.to_numeric(derecho_dispatchdata['Longitude1'])
derecho_dispatchdata['Latitude1'] = pd.to_numeric(derecho_dispatchdata['Latitude1'])
tornado_dispatchdata['Longitude1'] = pd.to_numeric(tornado_dispatchdata['Longitude1'])
tornado_dispatchdata['Latitude1'] = pd.to_numeric(tornado_dispatchdata['Latitude1'])

In [5]:
geometryD9C = [Point(xy) for xy in zip(derecho_911calls['ALI Longitude'], derecho_911calls['ALI Latitude'])]
geometryT9C = [Point(xy) for xy in zip(tornado_911calls['ALI Longitude'], tornado_911calls['ALI Latitude'])]
geometryDDD = [Point(xy) for xy in zip(derecho_dispatchdata['Longitude1'], derecho_dispatchdata['Longitude1'])]
geometryTDD = [Point(xy) for xy in zip(tornado_dispatchdata['Longitude1'], tornado_dispatchdata['Longitude1'])]

In [6]:
geo_derecho_911calls = gpd.GeoDataFrame(derecho_911calls, geometry = geometryD9C)
geo_tornado_911calls = gpd.GeoDataFrame(tornado_911calls, geometry = geometryT9C)
geo_derecho_dispatchdata = gpd.GeoDataFrame(derecho_dispatchdata, geometry = geometryDDD)
geo_tornado_dispatchdata = gpd.GeoDataFrame(tornado_dispatchdata, geometry = geometryTDD)

In [8]:
def changeByPct(geodf):
    num_rows = len(geodf.index)
    change_by = 1/num_rows
    return change_by
desired_hue = 360
hue_factor = (1/360)*desired_hue
base_point_color = Color(hue=hue_factor, saturation=1, luminance=0)

In [9]:
center = [36.1627,-86.7816]
map_nashville = folium.Map(location = center, zoom_start = 10, tiles="CartoDB positron")
change_by_base = changeByPct(geo_tornado_911calls)
change_by_sum = 0
current_point_color = base_point_color
geo_tornado_911calls = geo_tornado_911calls.sort_values('Seizure DateTime',ascending=False)
for ind, row in geo_tornado_911calls.iterrows():
    if math.isnan(row['ALI Latitude']):
        row['ALI Latitude'] = 0
    if math.isnan(row['ALI Longitude']):
        row['ALI Longitude'] = 0
    loc = [row['ALI Latitude'], row['ALI Longitude']]
    change_by_sum = change_by_sum + (change_by_base/1.5)
    current_point_color.luminance = change_by_sum
    if loc != [0,0]:
        folium.Circle(location = loc,radius=100,weight=4,color=current_point_color.hex, fill=True).add_to(map_nashville)
map_nashville

In [10]:
# Add a column with ones, then calculate sum and generate the heat
hm_gdf = geo_derecho_911calls['count'] = 1 
# create map of Calgary using latitude and longitude values
base_heatmap = folium.Map(location = center, zoom_start = 10, tiles="CartoDB positron")
HeatMap(data=geo_derecho_911calls[['ALI Latitude', 'ALI Longitude', 'count']].groupby(['ALI Latitude', 'ALI Longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=4).add_to(base_heatmap)
# To show the map in jupyter
base_heatmap

In [14]:
top10_incident_types = geo_derecho_dispatchdata.groupby('IncidentTypeDescription1').size().sort_values(ascending=False)
top10_incident_types = top10_incident_types.head(10).index.to_list()
top10gdf = geo_derecho_dispatchdata[geo_derecho_dispatchdata['IncidentTypeDescription1'].isin(top10_incident_types)]
incidentColors = ['#FE2712','#FB9902','#FCCC1A','#FEFE33','#66B032','#347C98','#0247FE','#4424D6','#8601AF','#C21460']
colorDict = dict(zip(top10_incident_types, incidentColors))
map_incident_type = folium.Map(location = center, zoom_start = 10, tiles="CartoDB positron")
for ind, row in top10gdf.iterrows():
    folium.Circle(location = [row['Latitude1'],row['Longitude1']],radius=100,weight=4,color=colorDict[row['IncidentTypeDescription1']], fill=True).add_to(map_incident_type)

map_incident_type

In [ ]:
#make 2 column df out of counts of each incident type
#stepping through each row